In [35]:
# Wind Capacity Prediction

In [36]:
import numpy as np
import pandas as pd
import xarray as xr
from shapely.geometry import Point
import matplotlib.pyplot as plt

paths = {"era5_eu_2013": "resources/europe-2013-era5.nc",
         "era5_tutorial": "resources/europe-2013-era5-tutorial.nc",
         "offshore_shape": "resources/regions_offshore_elec_s_37.geojson",
         "onshore_shape": "resources/regions_onshore_elec_s_37.geojson",
         "capfacs": "resources/capfacs_37.csv",
         "era5_regions": "resources/europe-2013-era5-regions.nc"}

In [37]:
ds = xr.open_dataset(filename_or_obj=paths["era5_regions"], engine="netcdf4")
ds

<xarray.Dataset>
Dimensions:           (region: 65, time: 8760)
Coordinates:
  * region            (region) object 'AL0 0 on' 'AT0 0 on' ... 'SI0 0 off'
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    height            (region, time) float64 ...
    wnd100m           (region, time) float64 ...
    roughness         (region, time) float64 ...
    influx_toa        (region, time) float64 ...
    influx_direct     (region, time) float64 ...
    influx_diffuse    (region, time) float64 ...
    albedo            (region, time) float64 ...
    temperature       (region, time) float64 ...
    soil_temperature  (region, time) float64 ...
    runoff            (region, time) float64 ...
Attributes:
    description:  Era5 data with mean value of the coordinates within a region

In [43]:
capfacts = pd.read_csv(paths["capfacs"])
capfacts

,snapshot,AL0 0 offwind-ac,AL0 0 onwind,AL0 0 solar,AT0 0 onwind,AT0 0 ror,AT0 0 solar,BA0 0 onwind,BA0 0 solar,BE0 0 offwind-ac,...,SE4 0 onwind,SE4 0 ror,SE4 0 solar,SI0 0 offwind-ac,SI0 0 onwind,SI0 0 ror,SI0 0 solar,SK0 0 onwind,SK0 0 ror,SK0 0 solar
0,2013-01-01 00:00:00,0.003291,0.001469,0.0,0.163262,0.224456,0.0,0.007340,0.0,1.000000,...,0.459609,0.626955,0.0,0.000000,0.055146,0.344668,0.0,0.361009,0.106197,0.0
1,2013-01-01 01:00:00,0.002103,0.000000,0.0,0.171340,0.224369,0.0,0.007939,0.0,0.999998,...,0.463265,0.625502,0.0,0.000000,0.052605,0.344657,0.0,0.368912,0.106012,0.0
2,2013-01-01 02:00:00,0.000000,0.000000,0.0,0.171035,0.224300,0.0,0.007829,0.0,0.993941,...,0.463777,0.624810,0.0,0.000000,0.052222,0.344593,0.0,0.382949,0.105968,0.0
3,2013-01-01 03:00:00,0.000000,0.000000,0.0,0.169685,0.224249,0.0,0.005766,0.0,0.916094,...,0.463041,0.623794,0.0,0.000000,0.050762,0.344626,0.0,0.388344,0.106215,0.0
4,2013-01-01 04:00:00,0.000000,0.000000,0.0,0.159757,0.224213,0.0,0.004262,0.0,0.704786,...,0.457253,0.623085,0.0,0.000000,0.047285,0.344607,0.0,0.409303,0.106364,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,0.029235,0.024321,0.0,0.109127,0.275353,0.0,0.001076,0.0,0.998922,...,0.358191,0.469586,0.0,0.027877,0.001708,0.696975,0.0,0.125735,0.157729,0.0
8756,2013-12-31 20:00:00,0.034024,0.031357,0.0,0.120485,0.275159,0.0,0.001090,0.0,0.965757,...,0.324709,0.469395,0.0,0.023567,0.000000,0.696262,0.0,0.142431,0.157817,0.0
8757,2013-12-31 21:00:00,0.037104,0.034522,0.0,0.128186,0.274975,0.0,0.001062,0.0,0.852484,...,0.288415,0.469208,0.0,0.000000,0.004891,0.695524,0.0,0.153930,0.157899,0.0
8758,2013-12-31 22:00:00,0.029730,0.027419,0.0,0.124740,0.274784,0.0,0.001351,0.0,0.776442,...,0.268695,0.469028,0.0,0.000000,0.009091,0.694665,0.0,0.163442,0.157945,0.0


In [42]:
capfacts.filter(regex=("DK.*"))

,DK0 0 offwind-ac,DK0 0 offwind-dc,DK0 0 onwind,DK0 0 solar,DK4 0 offwind-ac,DK4 0 offwind-dc,DK4 0 onwind,DK4 0 solar
0,0.952792,0.730188,0.874554,0.0,1.000000,1.000000,1.000000,0.0
1,0.882936,0.653330,0.764085,0.0,1.000000,1.000000,1.000000,0.0
2,0.772272,0.550946,0.600058,0.0,1.000000,1.000000,0.999901,0.0
3,0.707048,0.564461,0.489284,0.0,0.999319,1.000000,0.997682,0.0
4,0.665915,0.586840,0.416276,0.0,0.980715,1.000000,0.975385,0.0
...,...,...,...,...,...,...,...,...
8755,0.901924,0.988663,0.748381,0.0,0.797654,0.841022,0.721300,0.0
8756,0.919375,0.990556,0.759302,0.0,0.797028,0.842565,0.732721,0.0
8757,0.934671,0.994125,0.763535,0.0,0.808102,0.833833,0.764477,0.0
8758,0.958620,0.999466,0.786305,0.0,0.922194,0.916863,0.868329,0.0


## Countries for optimizing hyperparameters

Countries with the highest [installed wind power capacity](https://en.wikipedia.org/wiki/Wind_power_in_the_European_Union) in 2021:
1. Germany
2. Spain
3. UK
4. France
5. Sweden

Countries where wind covers the highest [percentage of the electricity demand](https://windeurope.org/wp-content/uploads/files/about-wind/statistics/WindEurope-Annual-Statistics-2019.pdf) in 2019:
1. Denmark
2. Ireland
3. Portugal
4. Germany
5. UK
6. Spain

In [39]:
ds["region"].values

array(['AL0 0 on', 'AT0 0 on', 'BA0 0 on', 'BE0 0 on', 'BG0 0 on',
       'CH0 0 on', 'CZ0 0 on', 'DE0 0 on', 'DK0 0 on', 'DK4 0 on',
       'EE6 0 on', 'ES0 0 on', 'ES2 0 on', 'FI4 0 on', 'FR0 0 on',
       'GB3 0 on', 'GB5 0 on', 'GR0 0 on', 'HR0 0 on', 'HU0 0 on',
       'IE5 0 on', 'IT0 0 on', 'IT1 0 on', 'LT6 0 on', 'LU0 0 on',
       'LV6 0 on', 'ME0 0 on', 'MK0 0 on', 'NL0 0 on', 'NO4 0 on',
       'PL0 0 on', 'PT0 0 on', 'RO0 0 on', 'RS0 0 on', 'SE4 0 on',
       'SI0 0 on', 'SK0 0 on', 'AL0 0 off', 'BE0 0 off', 'BG0 0 off',
       'DE0 0 off', 'DK0 0 off', 'DK4 0 off', 'EE6 0 off', 'ES0 0 off',
       'ES2 0 off', 'FI4 0 off', 'FR0 0 off', 'GB3 0 off', 'GB5 0 off',
       'GR0 0 off', 'HR0 0 off', 'IE5 0 off', 'IT0 0 off', 'IT1 0 off',
       'LT6 0 off', 'LV6 0 off', 'ME0 0 off', 'NL0 0 off', 'NO4 0 off',
       'PL0 0 off', 'PT0 0 off', 'RO0 0 off', 'SE4 0 off', 'SI0 0 off'],
      dtype=object)

### Denmark (DK)

- 08 DK0 0 on: 48 points
- 09 DK4 0 on: 10 points
- 41 DK0 0 off: 135 points
- 42 DK4 0 off: 38 points

In [40]:
ds.sel(region="DK0 0 on")

<xarray.Dataset>
Dimensions:           (time: 8760)
Coordinates:
    region            <U8 'DK0 0 on'
  * time              (time) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
Data variables:
    height            (time) float64 ...
    wnd100m           (time) float64 ...
    roughness         (time) float64 ...
    influx_toa        (time) float64 ...
    influx_direct     (time) float64 ...
    influx_diffuse    (time) float64 ...
    albedo            (time) float64 ...
    temperature       (time) float64 ...
    soil_temperature  (time) float64 ...
    runoff            (time) float64 ...
Attributes:
    description:  Era5 data with mean value of the coordinates within a region